In [ ]:
import os
import re
import json
from pathlib import Path
from tqdm.auto import tqdm

import pandas as pd
import torch
from transformers import BartForConditionalGeneration, BartTokenizer

CSV_FN = "disaster_articless.csv" 
df = pd.read_csv(CSV_FN, dtype=str).fillna("")

In [ ]:
# CELL 2 - Label province

# Provinces
provinces = [
    "Abra","Agusan del Norte","Agusan del Sur","Aklan","Albay","Antique","Apayao","Aurora",
    "Basilan","Bataan","Batanes","Batangas","Benguet","Biliran","Bohol","Bukidnon","Bulacan",
    "Cagayan","Camarines Norte","Camarines Sur","Camiguin","Capiz","Catanduanes","Cavite",
    "Cebu","Cotabato","Davao del Norte","Davao del Sur","Davao de Oro","Davao Occidental","Davao Oriental",
    "Dinagat Islands","Eastern Samar","Guimaras","Ifugao","Ilocos Norte","Ilocos Sur","Iloilo","Isabela",
    "Kalinga","La Union","Laguna","Lanao del Norte","Lanao del Sur","Leyte","Maguindanao del Norte", 
    "Maguindanao del Sur","Marinduque","Masbate","Metro Manila","Misamis Occidental","Misamis Oriental",
    "Mountain Province","Negros Occidental","Negros Oriental","Northern Samar","Nueva Ecija","Nueva Vizcaya",
    "Occidental Mindoro","Oriental Mindoro","Palawan","Pampanga","Pangasinan","Quezon","Quirino","Rizal",
    "Romblon","Samar","Sarangani","Siquijor","Sorsogon","South Cotabato","Southern Leyte","Sultan Kudarat",
    "Surigao del Norte","Surigao del Sur","Tarlac","Tawi-Tawi","Zambales","Zamboanga del Norte",
    "Zamboanga del Sur","Zamboanga Sibugay"
]

# Metro Manila city → province map
metro_manila_cities = [
    "Pasay", "Makati", "Quezon City", "Mandaluyong", "Caloocan",
    "Manila", "Navotas", "Taguig", "Parañaque", "Marikina", "Pasig",
    "Valenzuela", "Las Piñas", "Muntinlupa", "Pateros", "Malabon"
]
city_to_province = {
    "Quezon City": "Metro Manila",
    "Makati": "Metro Manila",
    "Manila": "Metro Manila",
    "Pasay": "Metro Manila",
    "Caloocan": "Metro Manila",
    "Davao City": "Davao del Sur",
    "Cagayan de Oro": "Misamis Oriental",
    # brute force manually add ICC, CC, and HUC
}

import re

def detect_provinces_hybrid(text):
    """
    Detect provinces where the disaster actually occurred,
    ignoring news source locations (e.g., Manila).
    """
    text_lower = text.lower()
    found_provinces = set()

    # Rule-based: explicit province names
    for prov in provinces:
        if prov.lower() in text_lower:
            found_provinces.add(prov)
    
    # Rule-based: city-to-province mapping, ignore source cities
    for city, prov in city_to_province.items():
        if "MANILA":
            continue  # skip news source mentions
        if city.lower() in text_lower:
            found_provinces.add(prov)
    
    # ML fallback if still empty
    if not found_provinces:
        found_provinces.add("Unknown")
    
    return list(found_provinces)



In [ ]:
# CELL 3 - load model and tokenizer (uses GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

MODEL_NAME = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(MODEL_NAME)
model = BartForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
model.eval()

# Summarization helper with chunking
def summarize_text(text, max_input_tokens=1024, max_summary_tokens=150, chunk_overlap=50):
    """
    Token-based chunking: split long input into chunks of max_input_tokens - overlap,
    summarize each chunk, then summarize concatenated chunk-summaries into one final summary.
    """
    if not text or text.strip() == "":
        return ""
    # Tokenize full text (list of token ids)
    tokens = tokenizer.encode(text, add_special_tokens=False)
    n = len(tokens)
    if n <= max_input_tokens:
        inputs = tokenizer.encode_plus(text, return_tensors="pt", truncation=True, max_length=max_input_tokens)
        input_ids = inputs["input_ids"].to(device)
        with torch.no_grad():
            summary_ids = model.generate(input_ids, num_beams=4, max_length=max_summary_tokens, early_stopping=True)
        return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    # chunk
    summaries = []
    stride = max_input_tokens - chunk_overlap
    for i in range(0, n, stride):
        chunk_ids = tokens[i : i + max_input_tokens]
        chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True)
        inputs = tokenizer.encode_plus(chunk_text, return_tensors="pt", truncation=True, max_length=max_input_tokens)
        input_ids = inputs["input_ids"].to(device)
        with torch.no_grad():
            summary_ids = model.generate(input_ids, num_beams=4, max_length=max_summary_tokens, early_stopping=True)
        chunk_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(chunk_summary)
        if i + max_input_tokens >= n:
            break
    # combine chunk summaries into final summary
    combined = " ".join(summaries)
    # final compress summarization (optional)
    inputs = tokenizer.encode_plus(combined, return_tensors="pt", truncation=True, max_length=max_input_tokens)
    input_ids = inputs["input_ids"].to(device)
    with torch.no_grad():
        final_ids = model.generate(input_ids, num_beams=4, max_length=max_summary_tokens, early_stopping=True)
    final_summary = tokenizer.decode(final_ids[0], skip_special_tokens=True)
    return final_summary


In [ ]:
# CELL 4 - disaster classification by keywords
disaster_type_keywords = {
    "flooding": ["flood", "flooding", "floods", "inundation", "water level", "submerged", "flash flood", "flooded"],
    "typhoon": ["typhoon", "super typhoon", "bagyo", "storm", "tropical storm", "landfall"],
    "earthquake": ["earthquake", "tremor", "aftershock", "magnitude", "richter"],
    "fire": ["fire", "blaze", "wildfire", "burned", "arson", "inferno", "smoke"],
    "volcanic eruption": ["volcano", "eruption", "ashfall", "lahar", "pyroclastic", "volcanic"],
}

# Normalize keywords to lowercase
for k in disaster_type_keywords:
    disaster_type_keywords[k] = [w.lower() for w in disaster_type_keywords[k]]

def classify_disaster_types(text):
    """
    Returns a list of disaster type labels detected in text using keyword matching.
    If none matched, returns ["other"].
    """
    text_l = (text or "").lower()
    found = set()
    for dtype, kws in disaster_type_keywords.items():
        for kw in kws:
            if kw in text_l:
                found.add(dtype)
                break
    if not found:
        # fallback: look for words like 'evacuation' or 'calamity' etc. to mark as 'other'
        if any(x in text_l for x in ["evacuate", "evacuee", "calamity", "disaster", "incident"]):
            return ["other"]
        return []
    return sorted(found)


In [ ]:
# CELL 5 - province detection helpers

def detect_provinces(text, provinces_list=provinces):
    """
    Returns a list of province names (original-cased) that appear in text.
    Uses approximate substring matching: tokenized words compared to province tokens.
    """
    t = (text or "").lower()
    matches = set()
    # exact substring match for province names
    for p in provinces:
        if p.lower() in t:
            matches.add(p)
    # additional heuristic: check common short forms (Metro Manila / Manila)
    if "manila" in t and "Metro Manila" in provinces:
        matches.add("Metro Manila")
    return sorted(matches)


In [ ]:
# CELL 6 - main processing
OUTPUT_JSON = "static/data/summaries_by_province.json"
OUTPUT_CSV = "static/data/summaries_flat.csv"

results = []   # flat list of processed records

# iterate rows (use tqdm)
for idx, row in tqdm(df.iterrows(), total=len(df)):
    article_text = row.get("Article") or row.get("Abstract") or row.get("Headline") or ""
    if not article_text or article_text.strip()=="":
        continue
    # optionally use Headline + Abstract + Article to give more context
    combined_text = " ".join([str(row.get(c,"")) for c in ["Headline","Abstract","Article"] if row.get(c)])
    # classify disaster types (keyword rules)
    types = classify_disaster_types(combined_text)
    provinces_found = detect_provinces_hybrid(combined_text)

    # summarize (skip if extremely short)
    summary = summarize_text(combined_text)
    
    record = {
        "index": int(idx),
        "Date": row.get("Date",""),
        "Headline": row.get("Headline",""),
        "Link": row.get("Link",""),
        "Detected_Provinces": provinces_found,
        "Detected_Types": types if types else ["other"],
        "Summary": summary
    }
    results.append(record)

# save flat CSV
pd.DataFrame(results).to_csv(OUTPUT_CSV, index=False, encoding="UTF-8")
print("Wrote flat results to", OUTPUT_CSV)


In [ ]:
# CELL 7 - aggregate
from collections import defaultdict

agg = defaultdict(lambda: defaultdict(list))
for r in results:
    for p in r["Detected_Provinces"]:
        for t in r["Detected_Types"]:
            agg[p][t].append({
                "Date": r["Date"],
                "Headline": r["Headline"],
                "Link": r["Link"],
                "Summary": r["Summary"]
            })

# Convert to plain dict for JSON serialization
agg_out = {p: {t: vals for t, vals in types.items()} for p, types in agg.items()}

with open(OUTPUT_JSON, "w", encoding="utf-8") as fh:
    json.dump(agg_out, fh, ensure_ascii=False, indent=2)

print("Wrote aggregated summaries to", OUTPUT_JSON)


In [ ]:
# CELL 8 - quick check and example display
import json
with open(OUTPUT_JSON, encoding="utf-8") as fh:
    data = json.load(fh)

# Show provinces that have entries
for prov, types in list(data.items())[:10]:
    print(prov, "->", {t: len(v) for t, v in types.items()})
